# MorphCT

goal: 
 - atomistic gsd snapshot -> 
 - assign chromophores -> 
 - do QCC calcs -> 
 - run KMC -> 
 - calculate mobility

current schema:
 - xml file
 - chromphore params set in par.py
 - if starting with atomistic, we can skip fine graining and molecular dynamics and only run:
     - execute_obtain_chromophores = False                                             
     - execute_ZINDO = False                                                           
     - execute_calculate_transfer_integrals = False                                    
     - execute_calculate_mobility = False    

In [1]:
from collections import defaultdict
import itertools
import os
import pickle
import multiprocessing as mp

import ele
import freud
import gsd.hoomd
import mbuild as mb
import numpy as np
from openbabel import openbabel
from openbabel import pybel
import pyscf
from pyscf.semiempirical import MINDO3

from morphct.code import obtain_chromophores as oc
from morphct import transfer_integrals as ti
from morphct import execute_qcc as eqcc
from morphct import mobility_kmc as kmc
from morphct.utils import kmc_analyze
from morphct import chromophores

/Users/jenny/miniconda3/envs/morphct-dev/lib/python3.8/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/Users/jenny/miniconda3/envs/morphct-dev/lib/python3.8/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` o

In [4]:
def visualize_qcc_input(qcc_input):
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()

OK, so I'm looking around for an xml file in the "obtain chromophores" tests but all I can find are these pickle files. I want to view them before I continue. ovito and vmd no longer support xmls... gah --> using mbuild.

In [5]:
path = "tests/assets/donor_polymer/OC/donor_polymer_post_obtain_chromophores.pickle"
(
    AA_morphdict, 
    CG_morphdict, 
    CGtoAAID_list, 
    param_dict, 
    chromo_list_old
) = pickle.load(open(path,"rb"))

In [6]:
all_types = list(set(AA_morphdict['type']))
bond_array = np.array([(i,j) for name,i,j in AA_morphdict["bond"]])

snap = gsd.hoomd.Snapshot()
snap.configuration.box = np.array([
    AA_morphdict["lx"],
    AA_morphdict["ly"],
    AA_morphdict["lz"],
    AA_morphdict["xy"],
    AA_morphdict["xz"],
    AA_morphdict["yz"]
])
snap.configuration.dimensions = AA_morphdict["dimensions"]
snap.particles.N = AA_morphdict["natoms"]
snap.particles.body = AA_morphdict["body"]
snap.particles.position = AA_morphdict["position"]
snap.particles.charge = AA_morphdict["charge"]
snap.particles.diameter = AA_morphdict["diameter"]
snap.particles.mass = AA_morphdict["mass"]
snap.particles.image = AA_morphdict["image"]
snap.particles.types = all_types
snap.particles.typeid = [all_types.index(i) for i in AA_morphdict["type"]]
snap.bonds.N = len(bond_array)
snap.bonds.group = bond_array
snap.validate()

In [7]:
unwrapped_positions = snap.particles.position + snap.particles.image * snap.configuration.box[:3]

In [8]:
# Run this if you want to confirm that the unwrapped positions are correct in OVITO
#with gsd.hoomd.open(name='test.gsd', mode='wb') as f:
#    f.append(snap)
#    snap.particles.position = unwrapped_positions
#    f.append(snap)

In [9]:
conversion_dict = {
    "S1": ele.element_from_symbol("S"), 
    "H1": ele.element_from_symbol("H"), 
    "C5": ele.element_from_symbol("C"), 
    "C1": ele.element_from_symbol("C"), 
    "C4": ele.element_from_symbol("C"), 
    "C6": ele.element_from_symbol("C"), 
    "C8": ele.element_from_symbol("C"), 
    "C9": ele.element_from_symbol("C"), 
    "C3": ele.element_from_symbol("C"), 
    "C7": ele.element_from_symbol("C"), 
    "C2": ele.element_from_symbol("C"),
    "C10": ele.element_from_symbol("C"),
}

amber_dict = {
    "c":  ele.element_from_symbol("C"),
    "c1": ele.element_from_symbol("C"),
    "c2": ele.element_from_symbol("C"),
    "c3": ele.element_from_symbol("C"),
    "ca": ele.element_from_symbol("C"),
    "cp": ele.element_from_symbol("C"),
    "cq": ele.element_from_symbol("C"),
    "cc": ele.element_from_symbol("C"),
    "cd": ele.element_from_symbol("C"),
    "ce": ele.element_from_symbol("C"),
    "cf": ele.element_from_symbol("C"),
    "cg": ele.element_from_symbol("C"),
    "ch": ele.element_from_symbol("C"),
    "cx": ele.element_from_symbol("C"),
    "cy": ele.element_from_symbol("C"),
    "cu": ele.element_from_symbol("C"),
    "cv": ele.element_from_symbol("C"),
    "h1": ele.element_from_symbol("H"),
    "h2": ele.element_from_symbol("H"),
    "h3": ele.element_from_symbol("H"),
    "h4": ele.element_from_symbol("H"),
    "h5": ele.element_from_symbol("H"),
    "ha": ele.element_from_symbol("H"),
    "hc": ele.element_from_symbol("H"),
    "hn": ele.element_from_symbol("H"),
    "ho": ele.element_from_symbol("H"),
    "hp": ele.element_from_symbol("H"),
    "hs": ele.element_from_symbol("H"),
    "hw": ele.element_from_symbol("H"),
    "hx": ele.element_from_symbol("H"),
    "f":  ele.element_from_symbol("F"),
    "cl": ele.element_from_symbol("Cl"),
    "br": ele.element_from_symbol("Br"),
    "i":  ele.element_from_symbol("I"),
    "n":  ele.element_from_symbol("N"),
    "n1": ele.element_from_symbol("N"),
    "n2": ele.element_from_symbol("N"),
    "n3": ele.element_from_symbol("N"),
    "n4": ele.element_from_symbol("N"),
    "na": ele.element_from_symbol("N"),
    "nb": ele.element_from_symbol("N"),
    "nc": ele.element_from_symbol("N"),
    "nd": ele.element_from_symbol("N"),
    "ne": ele.element_from_symbol("N"),
    "nf": ele.element_from_symbol("N"),
    "nh": ele.element_from_symbol("N"),
    "no": ele.element_from_symbol("N"),
    "o":  ele.element_from_symbol("O"),
    "oh": ele.element_from_symbol("O"),
    "os": ele.element_from_symbol("O"),
    "ow": ele.element_from_symbol("O"),
    "p2": ele.element_from_symbol("P"),
    "p3": ele.element_from_symbol("P"),
    "p4": ele.element_from_symbol("P"),
    "p5": ele.element_from_symbol("P"),
    "pb": ele.element_from_symbol("P"),
    "pc": ele.element_from_symbol("P"),
    "pd": ele.element_from_symbol("P"),
    "pe": ele.element_from_symbol("P"),
    "pf": ele.element_from_symbol("P"),
    "px": ele.element_from_symbol("P"),
    "py": ele.element_from_symbol("P"),
    "s":  ele.element_from_symbol("S"),
    "s2": ele.element_from_symbol("S"),
    "s4": ele.element_from_symbol("S"),
    "s6": ele.element_from_symbol("S"),
    "sh": ele.element_from_symbol("S"),
    "ss": ele.element_from_symbol("S"),
    "sx": ele.element_from_symbol("S"),
    "sy": ele.element_from_symbol("S"),
}

In [10]:
smarts_str = "c1cscc1CCCCCC"

In [11]:
aaids = chromophores.get_chromo_ids_smiles(snap, smarts_str, conversion_dict)

In [12]:
chromo_list = []
for i,aaid in enumerate(aaids):
    chromo_list.append(chromophores.Chromophore(i, snap, aaid, "donor", conversion_dict))

In [13]:
qcc_pairs = chromophores.set_neighbors_voronoi(chromo_list, snap, conversion_dict)

In [28]:
# Checked and all pairs look reasonable!
#visualize_qcc_input(qcc_pairs[25][1])

## Old method

In [14]:
sim_dims = [                                                                
    [-AA_morphdict["lx"] / 2.0, AA_morphdict["lx"] / 2.0],      
    [-AA_morphdict["ly"] / 2.0, AA_morphdict["ly"] / 2.0],      
    [-AA_morphdict["lz"] / 2.0, AA_morphdict["lz"] / 2.0],      
]   
chromo_list_old = oc.chromo_sort(chromo_list_old)
chromo_list_old = oc.determine_neighbours_voronoi(chromo_list_old, param_dict, sim_dims                          ) 

Calculating Neighbours of All Moieties
Updating the chromophore list for dissociation neighbours


In [37]:
for i in range(len(chromo_list)):
    print(f"chromo {i}:")
    print("\t      old        new")
    old_neighbors = [x for x,img in chromo_list_old[i].neighbours]
    for ind,img in chromo_list[i].neighbors:
        try:
            old_ind = old_neighbors.index(ind)
            print("\t", *chromo_list_old[i].neighbours[old_ind], ind, img)
        except ValueError:
            print(f"\tvalue err: {ind}")
            pass

chromo 0:
	      old        new
	 1 [0, 0, 0] 1 [0 0 0]
	 2 [0, 0, -1] 2 [0 0 0]
chromo 1:
	      old        new
	 0 [0, 0, 0] 0 [0 0 0]
	 2 [0, 0, 0] 2 [0 0 0]
	 3 [0, 0, 0] 3 [0 0 0]
chromo 2:
	      old        new
	 0 [0, 0, 1] 0 [0 0 0]
	 1 [0, 0, 0] 1 [0 0 0]
	 3 [0, 0, 0] 3 [0 0 0]
	 4 [0, 0, 0] 4 [0 0 0]
chromo 3:
	      old        new
	 1 [0, 0, 0] 1 [0 0 0]
	 2 [0, 0, 0] 2 [0 0 0]
	 4 [0, 0, 0] 4 [0 0 0]
chromo 4:
	      old        new
	 2 [0, 0, 0] 2 [0 0 0]
	 3 [0, 0, 0] 3 [0 0 0]
chromo 5:
	      old        new
	 6 [0, -1, 0] 6 [0 0 0]
chromo 6:
	      old        new
	 5 [0, 1, 0] 5 [0 0 0]
	 7 [0, 0, 0] 7 [0 0 0]
	value err: 8
chromo 7:
	      old        new
	 6 [0, 0, 0] 6 [0 0 0]
	 8 [0, 0, 0] 8 [0 0 0]
	 9 [0, 0, 0] 9 [0 0 0]
chromo 8:
	      old        new
	value err: 6
	 7 [0, 0, 0] 7 [0 0 0]
	 9 [0, 0, 0] 9 [0 0 0]
	 10 [0, 0, 0] 10 [0 0 0]
chromo 9:
	      old        new
	 7 [0, 0, 0] 7 [0 0 0]
	 8 [0, 0, 0] 8 [0 0 0]
	 10 [0, 0, 0] 10 [0 0 0]
	value err: 11
chromo 

In [55]:
for c in chromo_list_old:
    if not np.array_equal(c.image, np.zeros(3)):
        print(c.ID, c.image)

6 [0, -1, 0]
7 [0, -1, 0]
8 [0, -1, 0]
9 [0, -1, 0]
10 [0, -1, 0]
11 [0, -1, 0]
12 [0, -1, 0]
13 [0, -1, 0]
14 [0, -1, 0]
15 [0, -1, 0]
16 [0, -1, 0]
17 [0, -1, 0]
18 [0, -1, 0]
19 [0, -1, 0]
20 [0, -1, 0]
21 [0, -1, 0]
22 [0, -1, 0]


In [48]:
for c in chromo_list:
    if not np.array_equal(c.image, np.zeros(3)):
        print(c.id, c.image)

5 [0. 1. 0.]
6 [0. 1. 0.]
7 [0. 1. 0.]
8 [0. 1. 0.]
9 [0. 1. 0.]
10 [0. 1. 0.]
11 [0. 1. 0.]
12 [0. 1. 0.]
13 [0. 1. 0.]
14 [0. 1. 0.]
15 [0. 1. 0.]
16 [0. 1. 0.]
17 [0. 1. 0.]


In [69]:
d_cut = 10

box = snap.configuration.box[:3] 

chromo_i = chromo_list[17]
chromo_j = chromo_list[18]
centers = []                                                        
distances = []                                                      
images = []   


for xyz_image in itertools.product(range(-1,2), repeat=3):          
    xyz_image = np.array(xyz_image)                                 
    sc_center = chromo_j.center + xyz_image * box                   
    images.append(xyz_image)                                        
    centers.append(sc_center)                                       
    distances.append(np.linalg.norm(sc_center - chromo_i.center))   
imin = distances.index(min(distances))                              
if distances[imin] > d_cut:                                         
    pass   
rel_image = images[imin]                                            
j_shift = centers[imin] - chromo_j.unwrapped_center  

In [73]:
print(rel_image)
print(chromo_i.image, chromo_j.image)
print(chromo_i.center+ box*-rel_image, chromo_j.center)
print(chromo_i.unwrapped_center, chromo_j.unwrapped_center)
np.linalg.norm(chromo_i.unwrapped_center - chromo_j.unwrapped_center)

[0 1 0]
[0. 1. 0.] [0. 0. 0.]
[-22.23330602 -46.65447651 -16.1835895 ] [-25.56387659 -40.33081263 -10.96778323]
[-22.23330602 -46.65447651 -16.1835895 ] [-25.56387659 -40.33081263 -10.96778323]


8.847941017902542

In [68]:
ochromo_i = chromo_list_old[22]
ochromo_j = chromo_list_old[23]

rel_image = [0, 1, 0]

print(rel_image)
print(ochromo_i.image, ochromo_j.image)
print(ochromo_i.posn, ochromo_j.posn + box * rel_image)
print(ochromo_i.unwrapped_posn, ochromo_j.unwrapped_posn)


[0, 1, 0]
[0, -1, 0] [0, 0, 0]
[-20.7568099   42.30778292 -14.69372881] [-23.06243263  44.91209908 -12.37207311]
[-20.7568099  -42.88184568 -14.69372881] [-23.06243263 -40.27752952 -12.37207311]


In [ ]:
# These are different because geometric center vs CoM 
print(chromo_list[0].unwrapped_center,chromo_list_old[0].unwrapped_posn)

In [ ]:
print(dir(chromo_list_old[0])[:9])
print(dir(chromo_list_old[0])[35:])

In [ ]:
print(chromo_list_old[8].posn)
print(chromo_list_old[8].unwrapped_posn) 

# past here is outdated

Next time start working on obtain chromophores from snapshot

In [ ]:
#print(AA_morphdict.keys()) 
# 'xy', 'mass', 'lx', 'improper', 'body', 'unwrapped_position', 'natoms', 
# 'position', 'yz', 'xz', 'dimensions', 'ly', 'image', 'charge', 'lz', 'angle', 
# 'diameter', 'bond', 'time_step', 'type', 'dihedral'

#print(CG_morphdict.keys()) 
# same as above

#print(CGtoAAID_list) 
# {0: ['A', [0, 1, 2, 3, 4, 24]] includes attached hydrogen
# where A beads are thiophenes, B and C beads are first and second three alkyl carbons

#print(param_dict.keys())
# so many things...

#print(chromo_list[0])
# list of chromophore class objects

In [ ]:
#comp = mb.Compound()
#for name, pos in zip(AA_morphdict["type"],AA_morphdict["unwrapped_position"]):
#    name = name.strip("0123456789")
#    comp.add(mb.Particle(name=name, pos=np.array(pos)/10)) #convert to nm in mbuild
#ps = [p for p in comp.particles()]
#for _, i, j in AA_morphdict["bond"]:
#    comp.add_bond((ps[i],ps[j]))
#    
#comp.visualize().show()

OK, so we have 2 all-atom p3ht 15mers. 30 chromophores makes sense.

In [ ]:
print(len(chromo_list))
chromo = chromo_list[0]
print(dir(chromo))

In [ ]:
#print(len(chromo.AAIDs))
#for i in chromo.AAIDs:
#    ps[i].name = "x"
#comp.visualize().show()

Each chromophore is defined as one monomer.

All the HOMO,LUMO info has not been set, the file path doesn't exist, and the neighbors havent been set

In [ ]:
print(chromo.HOMO, chromo.LUMO)
print(chromo.species)
print(chromo.orca_output)
print(chromo.neighbours)

First let's fix the path -- we need to change the directories in the param_dict because they reference mattys computer. I'm making a new folder in the root dir called `notebook_output`. Then in that folder I had to make sure this dir structure exists in that folder:
```
/chromophores/ -+- input_orca/ -+- single/
                |               |
                |               +- pair/
                |
                +- output_orca/ -+- single/
                                 |
                                 +- pair/
```

*I have since change the code for this so it is no longer necessary*

In [ ]:
print(param_dict["output_orca_directory"])

In [ ]:
outpath = os.path.join(os.getcwd(),"notebook_output/")
print(outpath)
param_dict["output_orca_directory"] = outpath

Next let's get the neighbors

In [ ]:
# I changed the spelling because I am not British and it kept throwing me off
for chromo in chromo_list:
    chromo.neighbors = chromo.neighbours
    chromo.dissociation_neighbors = chromo.dissociation_neighbours
    chromo.neighbors_delta_E = chromo.neighbours_delta_E
    chromo.neighbors_TI = chromo.neighbours_TI
    chromo.pos = chromo.posn

In [ ]:
sim_dims = [                                                                
    [-AA_morphdict["lx"] / 2.0, AA_morphdict["lx"] / 2.0],      
    [-AA_morphdict["ly"] / 2.0, AA_morphdict["ly"] / 2.0],      
    [-AA_morphdict["lz"] / 2.0, AA_morphdict["lz"] / 2.0],      
]   
chromo_list = oc.chromo_sort(chromo_list)
chromo_list = oc.determine_neighbors_voronoi(                        
    chromo_list, param_dict, sim_dims                          
) 

The files are created by `morphct/code/execute_ZINDO.py` `create_input_files(chromophore_list, AA_morphology_dict, parameter_dict)`
The HOMO/LUMO gets set in `morphct/code/transfer_integrals.py` `load_orca_output(file_name)`

In [ ]:
qcc_pairs = eqcc.create_inputs(chromo_list, AA_morphdict, param_dict)
#print(qcc_pairs[0])
# (i,j), mol_str 

OK ~this writes 30 inputs but no pairs--some neighborlist analysis must need done first~ 

after neighbor list all files are written

next need to run ZINDO

`eqcc.get_homolumo(chromo_list[0].qcc_input)` returns HOMO-1, HOMO, LUMO, LUMO+1

In [ ]:
s_filename = os.path.join(outpath, "singles_energies.txt")
s_filename

In [ ]:
#%%time
#data = eqcc.singles_homolumo(chromo_list, s_filename)
#
#CPU times: user 17 ms, sys: 25.6 ms, total: 42.5 ms
#Wall time: 4.01 s

This gets the energy values of the chromophores using the single inputs. The neighbor energy values are not set:

In [ ]:
print(chromo.HOMO)
print(len(chromo.neighbours), len(chromo.neighbours_delta_E))
print(chromo.neighbours_delta_E[0])

next look in morphct/code/transfer_integrals.py

In [ ]:
d_filename = os.path.join(outpath, "dimer_energies.txt")
d_filename

In [ ]:
#%%time
#dimer_data = eqcc.dimer_homolumo(qcc_pairs, d_filename)
#
#CPU times: user 358 ms, sys: 146 ms, total: 504 ms
#Wall time: 2min 22s

In [ ]:
data = eqcc.get_singlesdata(s_filename)
print(data[0])
dimer_data = eqcc.get_dimerdata(d_filename)
print(dimer_data[0])

In [ ]:
eqcc.set_energyvalues(chromo_list, s_filename, d_filename)

In [ ]:
print(chromo.HOMO)
print(len(chromo.neighbours), len(chromo.neighbours_delta_E))
print(chromo.neighbours_delta_E[0])

OK, I think I should be ready to run KMC. Before it'll work we need to add some things to the param dict and change some paths.

```
notebook_outputs/KMC/ 
```

run_kmc : single_core_run_mob_KMC

kmc : mobility_KMC

In [ ]:
param_dict['simulation_times'] = [1.00e-13, 1.00e-12]
param_dict["number_of_holes_per_simulation_time"] = 10  
param_dict["number_of_electrons_per_simulation_time"] = 0 
param_dict["combine_KMC_results"] = True
param_dict["record_carrier_history"] = True
param_dict["hop_limit"] = 0
param_dict["system_temperature"] = 300 # In Kelvin
param_dict["output_morphology_directory"] = outpath

In [ ]:
jobs_list = kmc.get_jobslist(
    AA_morphdict, CG_morphdict, CGtoAAID_list, param_dict, chromo_list
)

In [ ]:
jobs_list[0]

In [ ]:
KMC_directory = os.path.join(outpath, "KMC")

In [ ]:
combined_data = kmc.run_kmc(
    jobs_list,                                                              
    KMC_directory,                                                          
    AA_morphdict,                                                           
    CG_morphdict,                                                           
    CGtoAAID_list,                                                          
    param_dict,                                                             
    chromo_list
)

In [ ]:
KMC_analyse.main(
    AA_morphdict, 
    CG_morphdict, 
    CGtoAAID_list, 
    param_dict,                                                             
    chromo_list,                                                            
    [combined_data],                                                      
    KMC_directory
)